In [1]:
import openslide
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import ast
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

In [2]:
!pip install h5py

In [3]:
df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')

In [4]:
df['label'] = df['label'].apply(lambda x: 1 if x == 'CE' else 0)

In [5]:
df = df.set_index('image_id')

In [6]:
samples = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

In [7]:
final_data= pd.read_csv('../input/final-preprocessing/navigation_v2.csv')
final_data = final_data.set_index('Unnamed: 0')
final_data = final_data.astype(object)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147,2148,2149,2150,2151,2152,2153,2154,2155,2156,2157,2158,2159,2160,2161,2162,2163,2164,2165,2166,2167,2168,2169,2170,2171,2172,2173,2174,2175,2176,2177,2178,2179,2180,2181,2182,2183,2184,2185,2186,2187,2188,2189,2190,2191,2192,2193,2194,2195,2196,2197,2198,2199,2200,2201,2202

In [8]:
def get_patch(path, tl_pixel, patch_shape):
    img = openslide.open_slide(path)
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return cv2.resize(img/255.0, (512, 512))

In [9]:
# def get_patch2(os_obj, tl_pixel, patch_shape):
#     return np.array(os_obj.read_region(tl_pixel, 0, patch_shape).convert("RGB"))

# final_data = pd.DataFrame()
# ids = []
# for n in range(df.shape[0]):
#     img_path = "../input/mayo-clinic-strip-ai/train/"+df.iloc[n]['image_id']+".tif"
#     label = df.iloc[n]['label']
#     i_id  = df.iloc[n]['image_id']
#     ids.append(i_id)
#     img  = openslide.open_slide(img_path)
#     series = pd.Series()
#     print(n)
#     for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#         for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#             im = get_patch2(img , (i,j), (512,512) )
#             if im.std()>15.0:
#                 s = pd.Series(data = [(i,j)])
#                 series = series.append(s,ignore_index=True)
#                 series = series.reset_index()
#                 series = series.drop("index",axis=1)
#     final_data = pd.concat([final_data , series],axis=1)
#     final_data.columns = ids


In [10]:
global train_test_list
train_test_list = np.random.choice(np.arange(754),754)

array([674, 624, 471,   0, 592,  84, 339, 704,  73, 706, 393,  66, 355,
       219, 249, 624, 132, 392, 375, 123, 146,  30, 151, 198, 217, 585,
       584, 520,  27, 148, 309, 351, 689, 395, 260, 485,  18, 674, 372,
       162, 550, 456, 126,  26, 225, 290, 367, 575, 493, 584, 372, 450,
       353, 396, 530,  25, 225, 195, 388, 673,  55,  44, 223, 119, 569,
       178, 655, 472, 403, 411, 703,  98, 234, 665, 526, 310,  83, 301,
       170, 415, 147, 479, 499, 668, 367, 498, 206, 428, 113, 477, 663,
       527, 600, 264,  24, 266, 355, 602,  96,  13, 423, 588, 108,  42,
       233, 219, 458, 388,  54,  98, 150, 620, 696, 622, 448, 628, 553,
       206, 214, 196, 308, 338,  39, 434, 368, 288,   2, 545, 311, 323,
       500, 745, 636, 653, 598, 470, 347, 525,  44, 249, 174, 302, 448,
       652, 386, 423, 447, 224, 716, 509, 305, 265, 369, 584, 212, 421,
       495, 526, 530, 526,  83, 661, 513, 108, 639, 752, 524, 471, 156,
       204, 575, 205, 355, 483, 665, 728, 741, 329, 239, 598,  3

In [11]:
def data_gen(df = df , data = final_data):
    for i in train_test_list[:650]:
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            for x in range(3):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            for x in range(1):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
       
    
    
def test_gen(df = df , data = final_data):
    for i in train_test_list[650:]:
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)

In [12]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)
testset = tf.data.Dataset.from_generator(
     test_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)

2022-09-25 19:24:12.477916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 19:24:12.601398: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 19:24:12.602681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 19:24:12.607376: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [13]:

dataset = dataset.batch(3)
dataset=dataset.prefetch(1)
dataset = dataset.shuffle(buffer_size = 10, seed=3)
testset = testset.batch(3)
testset=testset.prefetch(1)

In [28]:
efficient = tf.keras.applications.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_shape=(512,512,3),
)
def augmentation(A):
    A = tf.keras.layers.RandomFlip()(A)
    A =tf.keras.layers.RandomRotation(
        (-0.2, 0.2),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    A = tf.keras.layers.RandomZoom(
        (-0.1,0.1),
        (-0.1,0.1),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    return A

In [29]:
inputA = keras.layers.Input(shape=(512,512,3),name='input_71')
inputB = keras.layers.Input(shape=(512,512,3),name='input_72')
inputC = keras.layers.Input(shape=(512,512,3),name='input_73')
inputD = keras.layers.Input(shape=(512,512,3),name='input_74')
inputE = keras.layers.Input(shape=(512,512,3),name='input_75')

A = augmentation(inputA)
A = efficient(A)
A = keras.models.Model(inputs=inputA, outputs=A)
B = augmentation(inputB)
B = efficient(B)
B = keras.models.Model(inputs=inputB, outputs=B)
C = augmentation(inputC)
C = efficient(C)
C = keras.models.Model(inputs=inputC, outputs=C)
D = efficient(inputD)
D = keras.models.Model(inputs=inputD, outputs=D)
E = efficient(inputE)
E = keras.models.Model(inputs=inputE, outputs=E)

X = keras.layers.Concatenate(axis=-1)([A.output, B.output, C.output, D.output, E.output])
X = keras.layers.Conv2D(512, 3, padding="same")(X) 
X = keras.layers.Flatten()(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Dense(512 ,activation = 'relu')(X)
X = keras.layers.Dense(128 ,activation = 'relu')(X)
X = keras.layers.Dense(64 ,activation = 'relu')(X)
X = keras.layers.Dense(32 ,activation = 'relu')(X)
X = keras.layers.Dense(16 ,activation = 'relu')(X)
X = keras.layers.Dense(8 ,activation = 'relu')(X)
X = keras.layers.Dense(1 ,activation = 'sigmoid')(X)
                                               
model = keras.models.Model(inputs=[A.input,B.input,C.input,D.input,E.input], outputs=X)

In [30]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_72 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_73 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
random_flip_3 (RandomFlip)      (None, 512, 512, 3)  0           input_71[0][0]                   
___________________________________________________________________________________________

In [19]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.0001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.05,
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)

In [33]:
model.compile(optimizer=opt,
              loss=loss,
                metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), tf.keras.metrics.AUC()])

In [21]:
!mkdir model3

In [22]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './model3/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [23]:
H = model.fit(dataset, validation_data=testset, epochs=49, callbacks=[model_checkpoint_callback_LASSO])

Epoch 1/49


2022-09-25 19:25:47.108863: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-25 19:26:16.901390: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 5 of 10
2022-09-25 19:26:26.183360: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.
2022-09-25 19:26:27.136448: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


347/347 [==============================] - 792s 2s/step - loss: 0.7817 - recall_1: 0.3400 - precision_1: 0.5033 - auc_1: 0.5413 - val_loss: 0.8149 - val_recall_1: 0.2533 - val_precision_1: 0.6552 - val_auc_1: 0.4752
Epoch 2/49


2022-09-25 19:38:39.644941: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 6 of 10
2022-09-25 19:38:46.625931: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


347/347 [==============================] - 692s 2s/step - loss: 0.7400 - recall_1: 0.2595 - precision_1: 0.4085 - auc_1: 0.4928 - val_loss: 0.6190 - val_recall_1: 0.9333 - val_precision_1: 0.7609 - val_auc_1: 0.5629
Epoch 3/49


...

347/347 [==============================] - 669s 2s/step - loss: 0.6614 - recall_1: 0.3669 - precision_1: 0.6097 - auc_1: 0.6257 - val_loss: 0.5735 - val_recall_1: 1.0000 - val_precision_1: 0.7500 - val_auc_1: 0.5827
Epoch 48/49


2022-09-26 04:12:57.271674: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 6 of 10
2022-09-26 04:13:03.525393: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


347/347 [==============================] - 663s 2s/step - loss: 0.6519 - recall_1: 0.4385 - precision_1: 0.6087 - auc_1: 0.6578 - val_loss: 0.6743 - val_recall_1: 0.7333 - val_precision_1: 0.7971 - val_auc_1: 0.5704
Epoch 49/49


2022-09-26 04:24:19.401456: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 6 of 10
2022-09-26 04:24:25.863668: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


347/347 [==============================] - 645s 2s/step - loss: 0.6501 - recall_1: 0.4430 - precision_1: 0.6018 - auc_1: 0.6622 - val_loss: 0.6121 - val_recall_1: 1.0000 - val_precision_1: 0.7500 - val_auc_1: 0.5509


In [24]:
!mkdir final_model

In [25]:
!mkdir best_model

In [26]:
model.save('./final_model/model')

2022-09-26 04:36:19.045309: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [31]:
model.load_weights('./model3/model').expect_partial()
model.save('./best_model/model')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [34]:
model.evaluate(testset)

34/34 [==============================] - 66s 2s/step - loss: 0.5762 - recall_2: 1.0000 - precision_2: 0.7500 - auc_2: 0.4995


[0.5762194991111755, 1.0, 0.75, 0.4994666576385498]

In [ ]:
# def get_patch1(img, tl_pixel, patch_shape):
#     img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
#     return img

# final_samples= pd.DataFrame()
# ids = []
# for n in tqdm(range(samples.shape[0])):
#     img_path = "../input/mayo-clinic-strip-ai/test/"+samples.iloc[n]['image_id']+".tif"
#    #label = samples.iloc[n]['label']
#     i_id = samples.iloc[n]['image_id']
#     ids.append(i_id)
#     img  = openslide.open_slide(img_path)
#     series = pd.Series()
#     n = 0
#     for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#         for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#             if n ==20:
#                 break           
#             im = get_patch1(img , (i,j), (512,512) )
#             if im.std()>10.0:
#                 n=n+1
#                 s = pd.Series(data = [(i,j)])
#                 series = series.append(s,ignore_index=True)
#                 series = series.reset_index()
#                 series = series.drop("index",axis=1)
#     final_samples = pd.concat([final_samples , series],axis=1)
#     final_samples.columns = ids
# final_samples = final_samples.T


In [ ]:
# final_samples

In [ ]:
# samples

In [ ]:
# def sample_gen(df = samples , data = final_samples):
#     for i in range(samples.shape[0]):
#         img1 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][0]),(512,512))
#         img2 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][1]),(512,512))
#         img3 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][2]),(512,512))
#         img4 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][3]),(512,512))
#         img5 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][4]),(512,512))
#         yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5})
# sampleset = tf.data.Dataset.from_generator(
#      sample_gen,
#      ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}),
#     ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}))


In [ ]:
# weight_file = "./model"
# model.load_weights(weight_file).expect_partial()
# print("Weights loaded successfully")

In [ ]:
# weight_file = "./model"
# model.load_weights(weight_file).expect_partial()
# print("Weights loaded successfully")